In [37]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# Define the transformations for the dataset (you can modify this as needed)
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random color adjustments
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761])
])


# Download the CIFAR-100 training dataset
train_dataset = datasets.CIFAR100(
    root='./cifar-100-python',       # Change this path if needed
    train=True,          # Set to True to download the training set
    download=True,       # Set to True to download if not already downloaded
    transform=transform  # Apply transformations
)
##### Hyper-parameters
batch_size = 64 # 

# Create a DataLoader for batch processing
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,      # Batch size (you can modify this as needed)
    shuffle=True        # Shuffle data for training
)

# Example: Accessing one batch of images and labels
images, labels = next(iter(train_loader))
print(f"Batch of images shape: {images.shape}")
print(f"Batch of labels: {labels}")


Files already downloaded and verified
Batch of images shape: torch.Size([64, 3, 32, 32])
Batch of labels: tensor([88, 65, 10, 86, 15, 94, 21, 22, 80, 53, 42, 67, 59, 79, 85, 35, 57, 45,
        87, 97, 34, 85, 77, 74, 67,  8, 67, 96, 97,  4, 32, 18, 89, 61, 70, 16,
        31, 31, 52, 45, 26, 73, 38, 18, 21, 21, 55, 62, 94, 48, 97, 55, 48, 88,
        20, 43,  9, 71, 89, 18, 17, 41, 73, 81])


In [38]:
import torchvision
import torchvision.transforms as transforms

# Define transformations for the test set
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2761))  # CIFAR-100 mean and std
])

# Download the CIFAR-100 test dataset
test_dataset = torchvision.datasets.CIFAR100(
    root='./cifar-test-data',  # Specify the path where data will be stored
    train=False,    # Set to False to download the test set
    download=True,  # Automatically download the dataset if it's not present
    transform=transform
)

# Access the test data loader
test_loader = DataLoader(
    test_dataset, 
    batch_size=64,  # Set your preferred batch size
    shuffle=False   # Test set is typically not shuffled
)

print(f"Test set size: {len(test_dataset)} images")


Files already downloaded and verified
Test set size: 10000 images


In [39]:
import torch  # Core PyTorch library for tensor operations
from torchvision import datasets, transforms  # Datasets and transformations for computer vision
import torch.nn as nn  # Neural network components
import torch.optim as optim  # Optimization algorithms
import numpy as np  # Numerical operations
import matplotlib.pyplot as plt  # Plotting
from torch.utils.data import DataLoader, Dataset, random_split, Subset  # Data handling utilities

In [ ]:
batch_size = 64           # Batch size for data loading
num_epochs = 100          # Number of training epochs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [51]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define the model
class Cifar100Classifier(nn.Module):
    def __init__(self):
        super(Cifar100Classifier, self).__init__()
        self.cl1 = nn.Conv2d(3, 32, kernel_size=3, padding=1, stride=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.cl2 = nn.Conv2d(32, 64, kernel_size=3, padding=1, stride=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.cl3 = nn.Conv2d(64, 128, kernel_size=3, padding=1, stride=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.cl4 = nn.Conv2d(128, 256, kernel_size=3, padding=1, stride=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(256 * 2 * 2, 512)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(512, 100)

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool1(self.relu(self.bn1(self.cl1(x))))
        x = self.pool2(self.relu(self.bn2(self.cl2(x))))
        x = self.pool3(self.relu(self.bn3(self.cl3(x))))
        x = self.pool4(self.relu(self.bn4(self.cl4(x))))
        x = x.view(x.size(0), -1)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x
        

In [52]:
# Initialize model and move to device
model = Cifar100Classifier().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # CrossEntropy includes softmax
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=60)


In [53]:
def evaluate(model, data_loader, criterion):
    model.eval()
    correct = 0
    total = 0
    loss_batches = []

  # Switch to evaluation mode and turn off gradient calculation
  # since parameters are not updated during testing.
    with torch.no_grad():
        for images_batch, labels_batch in data_loader:
            outputs = model(images_batch)
            # The predicted label is the output with the highest activation.
            _, predicted = torch.max(outputs.data, 1)
            total += labels_batch.size(0)
            correct += (predicted == labels_batch).sum().item()

            # Use provided criterion to calculate the loss for the mini batch
            # Append the mini-batch loss to loss_batches array
            batch_loss = criterion(outputs, labels_batch)
            loss_batches.append(batch_loss.item())

    accuracy = 100 * correct / total
    loss = np.mean(loss_batches)

    model.train()

    return accuracy, loss

In [54]:
best_test_accuracy = 0.0  # Initialize the best accuracy

for epoch in range(num_epochs):
    for i, (images_batch, labels_batch) in enumerate(train_loader):
        optimizer.zero_grad()  # Clear the gradients
        outputs = model(images_batch)  # Forward pass
        loss = criterion(outputs, labels_batch)  # Calculate loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
        
    # Evaluate the model
    train_accuracy, train_loss = evaluate(model, train_loader, criterion)
    test_accuracy, test_loss = evaluate(model, test_loader, criterion)
    scheduler.step(test_loss)

    print(f'Epoch {epoch+1:02d}/{num_epochs:02d} - Train Loss: {train_loss:.6f}, Train Acc: {train_accuracy:.2f}%')
    print(f'            - Test Loss: {test_loss:.6f}, Test Acc: {test_accuracy:.2f}%')
    print("-" * 60)
    
    # Save model if test accuracy improves
    if test_accuracy > best_test_accuracy:
        best_test_accuracy = test_accuracy
        torch.save(model.state_dict(), 'best_model.pth')  # Save the model
        print(f"New best model saved with Test Acc: {best_test_accuracy:.2f}%")

print(f"Training complete. Best Test Accuracy: {best_test_accuracy:.2f}%")


c:\Users\MarkKogan\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\optim\lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 01/60 - Train Loss: 3.564788, Train Acc: 14.52%
            - Test Loss: 3.464212, Test Acc: 16.40%
------------------------------------------------------------
New best model saved with Test Acc: 16.40%
Epoch 02/60 - Train Loss: 3.299836, Train Acc: 18.05%
            - Test Loss: 3.194793, Test Acc: 19.77%
------------------------------------------------------------
New best model saved with Test Acc: 19.77%
Epoch 03/60 - Train Loss: 2.970397, Train Acc: 25.08%
            - Test Loss: 2.863430, Test Acc: 26.96%
------------------------------------------------------------
New best model saved with Test Acc: 26.96%
Epoch 04/60 - Train Loss: 2.742107, Train Acc: 29.66%
            - Test Loss: 2.663792, Test Acc: 30.88%
------------------------------------------------------------
New best model saved with Test Acc: 30.88%
Epoch 05/60 - Train Loss: 2.759492, Train Acc: 30.00%
            - Test Loss: 2.704371, Test Acc: 31.08%
------------------------------------------------------

In [55]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms

# Load test data
test_data = pd.read_csv('Given_CSV/test.csv')

ids = test_data['ID']
pixels = test_data.drop(columns=['ID']).values
pixels = pixels.reshape(-1, 3, 32, 32)

# Apply the reverse normalization to the pixel data
pixels_tensor = torch.stack([torch.tensor(img, dtype=torch.float32) for img in pixels])

# Create DataLoader
test_dataset = TensorDataset(pixels_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Step 4: Make predictions
model = Cifar100Classifier()
state_dict = torch.load("best_model.pth")  # Load the state_dict
model.load_state_dict(torch.load('best_model.pth', map_location=torch.device('cpu')))
        # Apply the weights to your model
model.eval()                              # Set to evaluation mode

predictions = []
with torch.no_grad():
    for batch in test_loader:
        inputs = batch[0].float()
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)  # Get the index of the max value for each prediction
        predictions.extend(predicted.cpu().numpy())

# Step 5: Prepare the result DataFrame
result_df = pd.DataFrame({
    'ID': ids,
    'LABEL': predictions
})

# Step 6: Save to a new CSV
result_df.to_csv('predictions.csv', index=False)



C:\Users\MarkKogan\AppData\Local\Temp\ipykernel_12616\1651938029.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("best_model.pth")  # Load the s

In [ ]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image

csv_path = './Given_CSV/test.csv'
df = pd.read_csv(csv_path)

print(f"Total samples in CSV: {len(df)}")

# Function to reverse the normalization
def unnormalize(tensor, mean, std):
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

cifar100_mean = (0.5071, 0.4867, 0.4408)
cifar100_std = (0.2675, 0.2565, 0.2761)

# Select a few samples to display (e.g., first 5)
num_samples = 10
samples = df.head(n=num_samples)  # Randomly select samples for diversity

# Iterate through the selected samples and display images
for index, row in samples.iterrows():
    # Extract pixel data and convert to numpy array
    pixel_data = row[[f'pixel_{i}' for i in range(1, 3073)]].values.astype(np.float32)

    # Reshape to (3, 32, 32)
    image = pixel_data.reshape(3, 32, 32)

    # Convert to torch tensor
    image_tensor = torch.tensor(image)

    # Reverse the normalization
    image_tensor = unnormalize(image_tensor, cifar100_mean, cifar100_std)

    # Clip the values to [0, 1] range
    image_tensor = torch.clamp(image_tensor, 0, 1)

    # Convert to numpy array and transpose to (32, 32, 3) for plotting
    image_np = image_tensor.numpy().transpose(1, 2, 0)

    # Convert to PIL Image for better handling (optional)
    image_pil = Image.fromarray((image_np * 255).astype(np.uint8))

    # Display the image using matplotlib
    plt.figure(figsize=(2,2))
    plt.imshow(image_pil)
    plt.axis('off')


In [ ]:
cifar100_label_names = [
    "apple", "aquarium_fish", "baby", "bear", "beaver", "bed", "bee", "beetle", "bicycle", "bottle", "bowl", "boy", "bridge", "bus", "butterfly", "camel", "can", "castle", "caterpillar", "cattle", "chair", "chimpanzee", "clock", "cloud", "cockroach", "couch", "crab", "crocodile", "cup", "dinosaur", "dolphin", "elephant", "flatfish", "forest", "fox", "girl", "hamster", "house", "kangaroo", "keyboard", "lamp", "lawn_mower", "leopard", "lion", "lizard", "lobster", "man", "maple_tree", "motorcycle", "mountain", "mouse", "mushroom", "oak_tree", "orange", "orchid", "otter", "palm_tree", "pear", "pickup_truck", "pine_tree", "plain", "plate", "poppy", "porcupine", "possum", "rabbit", "raccoon", "ray", "road", "rocket", "rose", "sea", "seal", "shark", "shrew", "skunk", "skyscraper", "snail", "snake", "spider", "squirrel", "streetcar", "sunflower", "sweet_pepper", "table", "tank", "telephone", "television", "tiger", "tractor", "train", "trout", "tulip", "turtle", "wardrobe", "whale", "willow_tree", "wolf", "woman", "worm"
]
label_mapping_df = pd.DataFrame({
    'LABEL': range(100),
    'Class Name': cifar100_label_names
})
desired_ids = [23,3,90,51,64,93,57,72]
filtered_df = label_mapping_df[label_mapping_df['LABEL'].isin(desired_ids)]

# Print the filtered rows
print(filtered_df)